**POZNÁMKA: Tento notebook je určený pre platformu Google Colab. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git
!{sys.executable} -m pip install umap_learn missingno

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
from class_utils import corr_heatmap, ColGrid, sorted_order, crosstab_plot, RainCloud
from umap import UMAP
import missingno as msno

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
DATA_HOME = "https://github.com/michalgregor/ml_notebooks/blob/main/data/{}?raw=1"

from class_utils.download import download_file_maybe_extract
download_file_maybe_extract(DATA_HOME.format("titanic.zip"), directory="data/titanic")

# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

## Exploratívna analýza dát

V tomto notebook-u sa budeme venovať niekoľkým základným spôsobom vizualizácie dáv Python-e- Vizualizácia je jedným z najsilnejších nástrojov **exploratívne analýzy dát**  (exploratory data analysis; EDA). Niekoľko vizualizácií sme už videli v automatických protokoloch, ktoré sme generovali v predošlom notebook-u. Tu sa pozrieme ešte na iné spôsoby ako vizualizovať dáta a zistiť, aké sú v nich zákonitosti.

Na vytvorenie grafov budeme používať balíček `seaborn` – silnú vizualizačnú knižnicu navrhnutú na použitie s balíčkom `pandas` – a `matplotlib` – populárnu vizualizačnú knižnicu, na ktorej `seaborn` stavia.

Znovu budeme pracovať s dátovou množinou Titanic.



In [ ]:
numeric_inputs = ["Pclass", "Age", "SibSp", "Parch", "Fare"]
categorical_inputs = ["Sex", "Embarked"]
ignored = ["PassengerId", "Name", "Ticket", "Cabin"]
output = "Survived"

df = pd.read_csv("data/titanic/train.csv")
df.head()

### Korelačná matica

Jednou z najužitočnejších vizualizácií – ktorú chceme typicky použiť hneď medzi prvými – je korelačná matica.

#### Klasická korelačná matica

Jednou možnosťou je použiť klasický druh korelačnej matice, ktorý používa farby a popisky na vizualizáciu korelácií medzi všetkými pármi numerických premenných. Hodnoty blízke nule znamenajú, že existuje len malá alebo žiadna lineárna závislosť medzi danými dvoma premennými. Vysoké pozitívne a negatívne korelácie sú obe zaujímavé: vysoká negatívna korelácia znamená, že medzi premennými je silná závislosť, ale sú si nepriamo úmerné.

Pre našu dátovú množinu by mohla klasická korelačná matica vyzerať takto:



In [ ]:
plt.figure(figsize=(10, 10))
corr_heatmap(df, map_type='standard', mask_diagonal=False);
plt.savefig("output/corr_unmasked.svg", bbox_inches="tight", pad_inches=0)

Všimnite si, že hodnoty na diagonálne budú vždy jednotky. Tieto môžu pôsobiť rušivo a – najmä ak sú iné korelácie skôr menšie – môžu dokonca narušiť aj farebnú škálu. Vo všeobecnosti je preto dobré diagonálu zamaskovať.



In [ ]:
plt.figure(figsize=(10, 10))
corr_heatmap(df, map_type='standard');
plt.savefig("output/corr.svg", bbox_inches="tight", pad_inches=0)

Korelačné koeficienty nám poskytujú predstavu o tom, aké lineárne závislosti existujú v našich dátach a aké sú silné. V prípade niektorých týchto korelácií však môže ísť len o náhodu. Ak máme k dispozícii konečné množstvo dát, väčšinou si nemôžeme byť 100% istí, že pozorované korelácie sú skutočné.

Môžeme však aspoň využiť nástroje, ktoré nám poskytuje štatistika a vypočítať **štatistickú významnosť**  týchto korelácií. Ak sa potom ukáže, že **p-hodnoty**  niektorých korelácií sú vysoké (je dobrá šanca, že sú tieto korelácie náhodné), môžeme ich v korelačnej matici zamaskovať.

Takto by naša korelačná matica vyzerala, keby sme v nej zamaskovali korelácie s p-hodnotami väčšími než 0.01. Všimnite si, že matica je teraz redšie obsadená, čo zlepšuje aj jej čitateľnosť.



In [ ]:
plt.figure(figsize=(10, 10))
corr_heatmap(df, map_type='standard', p_bound=0.01);
plt.savefig("output/corr_pbound_masked.svg", bbox_inches="tight", pad_inches=0)

#### Matica asociácií na spôsob SweetViz

Ak máme záujem o ešte lepší spôsob ako vizualizovať závislosti v dátovej množine, môžeme použiť maticu asociácií podobnú ako sa používa v rámci SweetViz protokolov. Tá rozširuje klasickú korelačnú maticu dvoma dôležitými spôsobmi:

* Vie zobraziť aj závislosti zahŕňajúce kategorické premenné:* Pomocou **korelačného pomeru**  (correlation ratio) pre numericko-kategorické interakcie;
* Pomocou **koeficientu neistoty**  (uncertainty coefficient) pre kategoricko-kategorické interakcie;

* Používa na kódovanie hodnôt okrem farieb aj rôzne tvary (kruhy pre numericko-numerické interakcie, obdĺžniky pre ostatné) a veľkosti (na znázornenie magnitúdy), takže je omnoho čitateľnejšia.
Všimnite si, že **koeficient neistoty**  je asymetrický. Ak však špecifikujete `sym_u=True`, získate jeho **symetrickú verziu** .



In [ ]:
plt.figure(figsize=(10, 10))
corr_heatmap(df, categorical_inputs=categorical_inputs);
plt.savefig("output/assoc.svg", bbox_inches="tight", pad_inches=0)

### Vizualizácia chýbania údajov

Balíček `missingno` poskytuje niekoľko zaujímavých druhov grafov, ktoré možno použiť na vizualizáciu chýbajúcich údajov. Dá sa pomocou neho rýchlo získať predstavu o tom, v ktorých stĺpcoch chýbajú údaje, koľko z nich chýba a dokonca aj to, či sú v spôsobe, ktorým dáta chýbajú, nejaké zaujímavé vzory.

#### Matica chýbajúcich údajov

Prvým z týchto grafov je matica chýbajúcich údajov, čo je súbor pruhov usporiadaných podľa riadkov a stĺpcov dátovej množiny. Biele pruhy označujú chýbajúce hodnoty, zatiaľ čo čierne pruhy označujú nechýbajúce hodnoty. Vpravo je graf, ktorý sumarizuje úplnosť/chýbanie celých riadkov – pomôže napr. nájsť riadky s maximálnym a minimálnym množstvom chýbajúcich dát.

Balíček obsahuje aj niekoľko ďalších druhov grafov, vrátane stĺpcového grafu chýbajúcich dát, grafu v štýle dendrogramu, ktorý zoskupuje stĺpce na základe korelácie ich vzorov chýbania atď.



In [ ]:
msno.matrix(df)
plt.savefig("output/missingness_matrix.svg", bbox_inches="tight", pad_inches=0)

Balíček obsahuje aj niekoľko ďalších druhov grafov, vrátane stĺpcového grafu chýbajúcich dát, grafu v štýle dendrogramu, ktorý zoskupuje stĺpce na základe korelácie ich vzorov chýbania atď.



In [ ]:
msno.bar(df)
plt.savefig("output/missingness_barplot.svg", bbox_inches="tight", pad_inches=0)

### Grafy rozdelení

Na preskúmanie každej premennej jednotlivo môžeme použiť rôzne druhy grafov rozdelení.

#### Rozdelenia numerických premenných

Pre jednotlivé numerické premenné sa rozdelenia typicky vizualizujú pomocou histogramov. Histogram rozdelí spojitú premennú na konečný počet diskrétnych intervalov (bins) a pomocou stĺpcového grafu vykreslí počet (alebo podiel) bodov spadajúci do každého z nich.

Nad histogramom sa často vizualizuje aj spojitá krivka (kernelový odhad hustoty; kernel density estimate; KDE), ktorá sa snaží aproximovať skutočné spojité rozdelenie pravdepodobnosti. Niekedy to zlepšuje čitateľnosť grafu.



In [ ]:
sns.histplot(x='Age', data=df, kde=True)
plt.savefig("output/hist.svg", bbox_inches="tight", pad_inches=0)

Počet intervalov sa dá typicky nastaviť, aby boli grafy podrobnejšie alebo naopak hrubšie. Intervaly sa takisto dajú špecifikovať aj manuálne.



In [ ]:
sns.histplot(x="Age", data=df, kde=True, bins=5)
plt.savefig("output/hist_5bin.svg", bbox_inches="tight", pad_inches=0)

Ak chceme tú istú vizualizačnú funkciu spustiť na viacerých stĺpcoch a zobraziť výsledky v mriežke, môžeme použiť `ColGrid` objekt a špecifikovať dátový rámec, stĺpce, ktoré sa majú použiť a počet stĺpcov v grafovej mriežke (`col_wrap`).



In [ ]:
g = ColGrid(df, numeric_inputs, col_wrap=2)
g.map_dataframe(sns.histplot, kde=True);
plt.savefig("output/hist_colgrid.svg", bbox_inches="tight", pad_inches=0)

#### Rozdelenia kategorických premenných: stĺpcové grafy

Pre kategorické premenné sa dajú namiesto histogramov použiť jednoduché **stĺpcové grafy**  (pomocou `sns.countplot`). Tieto sú histogramom podobné, ale už nie je potrebné deliť premennú na intervaly, keďže je aj sama osebe diskrétna.



In [ ]:
sns.countplot(x="Embarked", data=df)
plt.savefig("output/count.svg", bbox_inches="tight", pad_inches=0)

Znovu môžeme použiť `ColGrid` na to, aby sme ten istý typ grafu vytvorili pre všetky kategorické premenné.



In [ ]:
g = ColGrid(df, categorical_inputs + [output], col_wrap=2)
g.map_dataframe(sns.countplot);
plt.savefig("output/count_colgrid.svg", bbox_inches="tight", pad_inches=0)

##### Použtie argumentu `hue` v rámci `countplot`

Stĺpcové grafy vytvorené pomocou `countplot` (a podobne aj väčšina ostatných `seaborn` grafov) akceptujú argument `hue`, ktorý sa dá použiť na rozdelenie jedného stĺpca do viacerých rôznofarebných stĺpcov podľa niektorej diskrétnej premennej.

Môžeme napríklad znovu vykresliť počty naprieč rôznymi hodnotami "embarked", ale ich tento raz rozdeliť podľa triedy pasažiera. Z výsledného grafu sa dozvieme napríklad to, že väčšina pasažierov z tretej triedy sa nalodila v Southamptone.



In [ ]:
sns.countplot(x="Embarked", hue="Pclass", data=df)
plt.savefig("output/count_hue.svg", bbox_inches="tight", pad_inches=0)

### Grafy interakcií

Ďalšou vecou, ktorú môžeme vizualizovať sú rôzne spôsoby interakcie dvojíc premenných.

#### Numericko-numerické: bodový diagram

Na vykreslenie závislosti dvoch numerických stĺpcov sa dá použiť bodový diagram, kde na každú os pripadne jeden zo stĺpcov a každému bodu v grafe zodpovedá jeden riadok.



In [ ]:
sns.scatterplot(x='Age', y='Fare', data=df)
plt.savefig("output/scatter.svg", bbox_inches="tight", pad_inches=0)

Vytvorme si teraz bodové diagramy pre všetky kombinácie numerických stĺcov. Znovu použijeme `ColGrid` a špecifikujeme argument `interact='comb'`. Tým sa vyhneme vytvoreniu duplicitných grafov – nechceme napr. zobraziť `Age` vs. `Fare` a zároveň `Fare` vs. `Age` – to by bolo zbytočné.



In [ ]:
g = ColGrid(df, numeric_inputs, interact='comb', col_wrap=3)
g.map_dataframe(sns.scatterplot);
plt.savefig("output/scatter_colgrid.svg", bbox_inches="tight", pad_inches=0)

##### Ordinálne kategorické premenné

Všimnite si, že v našom prípade sú stĺpce `Age` a `Fare` jediné spojité numerické stĺpce. Ostatné numerické stĺpce sú diskrétne a každý z nich obsahuje len malý počet rozličných hodnôt. Ako uvidíme neskôr, niektoré z nich sa budú lepšie vizualizovať ak ich budeme chápať ako **ordinálne kategorické premenné**  (ordinal categorical variables).

Ordinálne kategorické premenné predstavujú špeciálny typ kategorických premenných, ktorých hodnoty sú určitým spôsobom prirodzene **usporiadané** . Napr. pri kategorickej premennej `výška` s hodnotami `nízky`, `stredný`, `vysoký`, je zrejmé, že `nízky` je v určitom zmysle najnižšia hodnota a `vysoký` najvyššia: napriek tomu, že je premenná kategorická.

Zaznamenajme si teraz, ktoré z našich numerických stĺpcov by sa dali namiesto toho charakterizovať ako ordinálne kategorické premenné a môžeme ich vyskúšať vizualizovať iným spôsobom v nasledujúcej sekcii.



In [ ]:
ordinal_inputs = ["Pclass", "SibSp", "Parch"]

#### Numericko-numerické: Regresný diagram

Existuje špeciálny druh grafu, ktorý kombinuje bodový graf s lineárnym regresným grafom. V niektorých prípadoch to uľahčuje čítanie grafu – je jasnejšie, aký lineárny trend môže byť v údajoch. Pre regresnú priamku sa vyfarbenou oblasťou zvyčajne vizualizujú aj intervaly spoľahlivosti.



In [ ]:
sns.regplot(x='SibSp', y='Fare', data=df)
plt.savefig("output/regplot.svg", bbox_inches="tight", pad_inches=0)

Ako vidno, premenná SibSp je síce číselná, ale diskrétna. Vďaka tomu je graf dosť ťažko čitateľný – jedna vec, ktorú môžeme v takýchto prípadoch urobiť, je pridať k bodom trochu jitteru. To znamená, že ku každému bodu pripočítame trochu náhodného šumu: v našom prípade do x-ovej súradnice každého bodu, pretože to je os, na ktorej je diskrétna premenná. Vďaka tomu budú body rozptýlené a graf ľahšie čitateľný. Body môžeme okrem toho spraviť čiastočne transparentné: to nám tiež poskytne lepšiu predstavu o tom, koľko bodov je v každej oblasti.



In [ ]:
sns.regplot(
    x='SibSp', y='Fare', data=df,
    x_jitter=0.25, scatter_kws={'alpha': 0.25}
)
plt.savefig("output/regplot_jitter_alpha.svg", bbox_inches="tight", pad_inches=0)

Funkcia regplot je tiež pohodlný spôsob, ako vytvoriť štandardné bodové grafy s jitterom, pretože regresná priamka sa dá vypnúť.



In [ ]:
sns.regplot(
    x='SibSp', y='Fare', data=df,
    x_jitter=0.25, scatter_kws={'alpha': 0.25},
    fit_reg=False
)
plt.savefig("output/regplot_jitter_alpha_noreg.svg", bbox_inches="tight", pad_inches=0)

#### Numericko-numerické: čiarový graf

Jedným z najznámejších druhov grafov je klasický čiarový graf. Keď však vytvárame graf tohto typu, musíme dbať na to, aby boli body správne usporiadané (inak sa čiary nemusia spojiť správnym spôsobom) a aby sme pre každú hodnotu na horizontálnej osi vykresľovali iba jednu hodnotu na vertikálnej. V bodovom diagrame sme vedeli nakresliť toľko hodnôt, koľko sme chceli: v prípade čiarového grafu by to nedávalo zmysel.

##### Správne usporiadanie

Aby sme problém ilustrovali, vytvorme teraz naivným spôsobom čiarový graf pre premenné 'Age' a 'Fare':



In [ ]:
df.plot.line(x='Age', y='Fare')

Graf vyzerá príšerne, pretože body sú spojené v takom poradí, v akom idú za sebou v dátovej množine. Aby sa spojili korektne, treba ich najprv zotriediť:



In [ ]:
df.sort_values(by='Age').plot.line(x='Age', y='Fare')

##### Priemerovanie

Výsledkom stále nie je dobrý graf, pretože sme ignorovali fakt, že pre mnohé hodnoty premennej 'Age' máme viacero hodnôt 'Fare'. Čo by sme mali spraviť je najprv dáta zoskupiť podľa veku a vypočítať priemery zodpovedajúcich hodnôt cestovného. Potom už bude výsledkom korektný graf. Všimnite si tiež, ako sa dva odľahlé body, kde bolo cestovné nad 500 priemerovaním zahladili.



In [ ]:
df[['Age', 'Fare']].groupby(by='Age').mean().plot.line()

##### Intervaly spoľahlivosti a vyhladzovanie

Tento graf stále nie je ideálny, pretože na rozdiel od bodového diagramu, nemáme predstavu aký bol v hodnotách cestovného pre jednotlivé veky rozptyl. To vieme vyriešiť zobrazením **intervalov spoľahlivosti**  (confidence intervals). Seaborn to realizuje automaticky (pomocou bootstrapping-u): vyfarbená plocha zodpovedá 95-percentným intervalom spoľahlivosti. Pravdepodobnosť, že skutočná stredná hodnota leží vnútri intervalu je 95%.



In [ ]:
sns.lineplot(x='Age', y='Fare', data=df)
plt.savefig("output/line.svg", bbox_inches="tight", pad_inches=0)

Pre veľmi zašumené dáta môže byť dobré zobraziť nad grafom aj jeho vyhladenú verziu. Napr. pomocou kĺzavého priemeru:



In [ ]:
sns.lineplot(x='Age', y='Fare', data=df)
df_grouped = df[['Age', 'Fare']].groupby(by='Age').mean()
moving_average = df_grouped.rolling(window=10, min_periods=1).mean()
moving_average.plot.line(ax=plt.gca(), linewidth=4)
plt.legend(['original fare', 'moving average'])
plt.savefig("output/line_ma.svg", bbox_inches="tight", pad_inches=0)

#### Kategoricko-kategorické: krížová tabulácia

Na vizualizáciu interakcie medzi dvoma kategorickými premennými môžeme pre ne vytvoriť krížovú tabuľku a vykresliť ju ako maticu. Každá bunka matice zodpovedá počtu spoločných výskytov zodpovedajúcich dvoch hodnôt v dátovej množine.



In [ ]:
crosstab_plot(x='Sex', y='Survived', data=df);
plt.savefig("output/crosstab.svg", bbox_inches="tight", pad_inches=0)

Tento graf jasne indikuje, že existuje veľmi silná asociácia medzi mužským pohlavím cestujúceho a jeho úmrtím na Titanicu. Existuje tiež dosť silná asociácia medzi ženským pohlavím a prežitím. A to nám už sprostredkuje dosť veľa informácií.

Aby sme získali interakcie každého kategorického vstupu s výstupnou premennou, použijeme 2-argumentovú verziu `ColGrid`. Do grafu môžeme zahrnúť aj ordinálne kategorické premenné, ktoré sme vyššie identifikovali.



In [ ]:
g = ColGrid(df, categorical_inputs + ordinal_inputs, output, col_wrap=2)
g.map_dataframe(crosstab_plot);
plt.savefig("output/crosstab_colgrid.svg", bbox_inches="tight", pad_inches=0)

#### Numericko-kategorické

Pre prípad numericko-kategorických interakcií sa budeme venovať dvom druhom grafov. Oba sú v podstate dosť podobné, ale každý z nich má vlastné silné stránky.

##### Box plot (krabicový diagram)

Znamejší z týchto dvoch typov grafov je box plot (krabicový diagram), ktorý vizualizuje rozdelenie numerickej premennej naprieč rôznymi hodnotami kategorickej premennej. Graf zobrazuje obdĺžniky (krabice; boxes) a úsečky (fúzy; whiskers). Obdĺžniky siahajú od 25. po 75. percentil (t.j. prostredných 50% dát leží vnútri obdĺžnika). Čiara pretínajúca obdĺžnik predstavuje medián (t.j. 50% dát leží pod touto čiarou).

Keďže obdĺžnik siaha od 1. kvartilu (25-teho percentilu) po 3. kvartil (75-ty percentil), výška obdĺžnika predstavuje medzikvartilové rozpätie (interquartile range; IRQ). Úsečky siahajú od minimálnej po maximálnu hodnotu z dátovej množiny, ale nie ďalej než po 1,5-krát IRQ (v dolnej časti od 1. kvartilu a v hornej od 3. kvartilu). Body mimo tohto rozsahu (ak také sú) sa považujú za odľahlé hodnoty (outliers) a zobrazujú sa samostatne.



In [ ]:
sns.boxplot(x="Sex", y="Age", data=df)
plt.savefig("output/box.svg", bbox_inches="tight", pad_inches=0)

Aby boli box plot-y čitateľnejšie, najmä keď obsahujú mnoho kategórií, býva často lepšie ak sa kategórie zoradia podľa mediánov. Presne na ten účel máme vytvorenú pomocnú funkciu `sorted_order`:



In [ ]:
sorted_order(sns.boxplot)(x="Sex", y="Age", data=df)
plt.savefig("output/box_sorted.svg", bbox_inches="tight", pad_inches=0)

##### Violin plot (husličkový  graf)

Violin plot-y (husličkové grafy) sú v niečom podobné box plot-om, poskytujú však úplnejšiu predstavu o tom ako vyzerá rozdelenie numerickej premennej. Podobajú sa v podstate rotovaným grafom hustoty (density plot-om vytvoreným pomocou kernel density estimation; KDE): hrúbka husličiek reprezentuje početnosť zodpovedajúcich numerických hodnôt. Violin plot-y sú osobitne užitočné ak je rozdelenie multimodálne (t.j. má viacero lokálnych maxím), čo sa v box plot-e nedá vizualizovať.

**Poznámka: Aby ste husličkový graf nečítali nesprávne, všimnite si, že husličky majú v hornej a spodnej časti vynechaný kúsok prázdneho priestoru. Je to tak kvôli vyhladzovaniu pomocou kernel density estimation (KDE). V dôsledku toho však minimálnu a maximálnu hodnotu nereprezentujú body, kde končia husličky, ale úsečka zobrazená vo vnútri husličiek.**  Porovnajte si husličkový graf s box plot-om, aby ste o tom získali úplnejšiu predstavu. Ak by ste chceli husličky orezať tak, aby nesiahali za minimum a maximum, môžete funkcii `violinplot` poslať argument `cut=0`, nie je to však predvolené správanie.

Vovnútri husličiek sa typicky zobrazuje malý box plot, kde obdĺžnik je nahradený hrubou čiarou, úsečky tenkou čiarou a medián sa zobrazuje ako biely krúžok.



In [ ]:
sorted_order(sns.violinplot)(x="Sex", y="Age", data=df)
plt.savefig("output/violin.svg", bbox_inches="tight", pad_inches=0)

Box plot-y aj violin plot-y môžeme prirodzene opať zobraziť v mriežke pomocou `ColGrid`.



In [ ]:
g = ColGrid(df, categorical_inputs, "Age", col_wrap=2)
g.map_dataframe(sorted_order(sns.violinplot));
plt.savefig("output/violin_categorical.svg", bbox_inches="tight", pad_inches=0)

##### Violin plot-y pre ordinálne kategorické premenné

To isté je možné vykresliť pre ordinálne kategorické premenné. Výsledné vizualizácie budú omnoho informatívnejšie než bodové diagramy, ktoré sme vytvorili vyššie. Všimnite si však, že ordinálne premennú majú už hodnoty prirodzene zoradené a preto by nebolo v ich prípade zmysluplné použiť `sorted_order` na ich zotriedenie.



In [ ]:
g = ColGrid(df, ordinal_inputs, "Age", col_wrap=2)
g.map_dataframe(sns.violinplot);
plt.savefig("output/violin_ordinal.svg", bbox_inches="tight", pad_inches=0)

##### Raincloud graf

Ak by ste chceli vytvoriť ešte sofistikovanejší graf – ktorý je však zároveň aj skutočne informatívnejší – môžete zostrojiť tzv. **raincloud graf** : nový, nedávno publikovaný typ grafu, ktorý kombinuje 3 aspekty:

* (polovičný) husličkový graf;
* boxplot a;
* zobrazenie reálnych dát.
Týmto spôsobom je možné získať dobrú vizuálnu sumarizáciu, ktorá poskytne už na prvý pohľad všetky najdôležitejšie informácie.

Viac o raincloud grafoch nájdete na [Raincloud Plots GitHub-e](https://github.com/RainCloudPlots/RainCloudPlots) alebo v samotnom článku:
Allen M, Poggiali D, Whitaker K et al. Raincloud plots: a multi-platform tool for robust data visualization [version 2; peer review: 2 approved]. Wellcome Open Res 2021, 4:63. DOI: 10.12688/wellcomeopenres.15191.2

O pokročilejších spôsoboch použitia nájdete viac v [raincloud_tutorial_python.ipynb](https://github.com/pog87/PtitPrince/blob/master/tutorial_python/raincloud_tutorial_python.ipynb).



In [ ]:
RainCloud(x="Sex", y="Age", data=df)
plt.savefig("output/raincloud.svg", bbox_inches="tight", pad_inches=0)

Alebo s použitím `ColGrid`:



In [ ]:
g = ColGrid(df, categorical_inputs, "Age", col_wrap=2)
g.map_dataframe(RainCloud);

### Facet mriežky

Facet mriežky, ktoré seaborn poskytuje, fungujú podobne ako ColGrid mriežky, ktoré sme už používali. Hlavný rozdiel je v tom, že ColGrid mriežky iterujú cez kombinácie stĺpcov, zatiaľ čo facet mriežky iterujú po rôznych hodnotách tej istej diskrétnej premennej, prípadne dvoch diskrétnych premenných. Vo facet mriežke sa dajú vykresliť všetky možné typy grafov, ktorým sme sa venovali.

#### Rozdelenie veku pre rôzne triedy

Ako príklad by sme si mohli vizualizovať histogramy veku `Age` naprieč rôznymi triedami cestujúcich a porovnať ich.



In [ ]:
g = sns.FacetGrid(df, col="Pclass")
g.map_dataframe(sns.histplot, x="Age", kde=True)
g.set_axis_labels("Age", "Count")
plt.savefig("output/facet.svg", bbox_inches="tight", pad_inches=0)

##### Automatické zalamovanie stĺpcov

Keby sme ten istý graf chceli vytvoriť pre stĺpec "SibSp", rozličných hodnôt by bolo priveľa na to, aby sa zmestili do kompaktného grafu. Našťastie je znovu možnú použiť argument `col_wrap` na automatické zalomenie stĺpcov do viacerých riadkov.



In [ ]:
g = sns.FacetGrid(df, col="SibSp", col_wrap=4)
g.map_dataframe(sns.histplot, x="Age", kde=True)
g.set_axis_labels("Age", "Count")
plt.savefig("output/facet_wrap.svg", bbox_inches="tight", pad_inches=0)

##### 2-rozmerné facet mriežky

Takisto je možné vytvoriť 2-rozmerné facet mriežky, kde stĺpce predstavujú jednu diskrétnu premennú a riadky druhú. V stĺpcoch by sa napríklad mohla meniť premenná "Pclass" a v riadkoch premenná "Embarked".



In [ ]:
g = sns.FacetGrid(df, col="Pclass", row="Embarked")
g.map_dataframe(sns.histplot, x="Age", kde=True)
g.set_axis_labels("Age", "Count")
plt.savefig("output/facet_2d.svg", bbox_inches="tight", pad_inches=0)

##### Iné typy grafov

Skutočná sila `FacetGrid` je samozrejme v tom, že podobne ako pri `ColGrid` funguje s ľubovoľným typom grafu, napr.:



In [ ]:
g = sns.FacetGrid(df, col="Pclass", row="Embarked")
g.map_dataframe(sns.countplot, x="Survived")
g.set_axis_labels("Survived", "Count")
plt.savefig("output/facet_count.svg", bbox_inches="tight", pad_inches=0)

In [ ]:
g = sns.FacetGrid(df, col="Pclass", row="Embarked")
g.map_dataframe(sorted_order(sns.violinplot), x="Survived", y="Age")
g.set_axis_labels("Survived", "Age")
plt.savefig("output/facet_violin.svg", bbox_inches="tight", pad_inches=0)

### Koláčové grafy pomocou Pandas

Napokon ešte spomenieme, že hoci `seaborn` nepodporuje koláčové grafy, balíček `pandas` áno. Keby sme si chceli zobraziť podiel medzi mužským a ženským pohlavím v dátovej množine, mohli by sme spočítať výskyty pre "Sex" pomocou `df["Sex"].value_counts()` a následne výsledky vykresliť pomocou `.plot.pie`. Môžeme špecifikovať `autopct='%1.0f%%'` aby sa v grafe zobrazili percentá a použiť argument `explode` na vysunutie niektorých častí koláča smerom von.



In [ ]:
df["Sex"].value_counts()

In [ ]:
df["Sex"].value_counts().plot.pie(autopct='%1.0f%%', explode=[0, 0.05]);
plt.savefig("output/pie.svg", bbox_inches="tight", pad_inches=0)

Keďže tieto grafy nedisponujú štandardným `seaborn` rozhraním, nedajú sa použiť spolu s `ColGrid` alebo `FacetGrid`. Stále sa však samozrejme dajú zobraziť v mriežke pomocou funkcie `subplots` z balíčka `matplotlib`, čo je trochu prácnejšie, ale funguje to rovnako dobre.



In [ ]:
# the columns to plot
cols_to_plot = categorical_inputs + [output]

# number of columns and rows in the subplots grid
ncols=3
nrows=int(np.ceil(len(cols_to_plot)/ncols))
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(12, 8))

# plot each column in a separate subplot
for col, ax in zip(cols_to_plot, np.ravel(axes)):
    df[col].value_counts().plot.pie(autopct='%1.0f%%', ax=ax)

# to hide unused axes
for ax in np.ravel(axes)[len(cols_to_plot):]:
    ax.axis('off')
    
plt.savefig("output/pie_subplots.svg", bbox_inches="tight", pad_inches=0)

**Upozorňujeme, že použitie koláčových grafov sa vo všeobecnosti neodporúča, pretože sú ťažšie čitateľné než stĺpcové grafy (ťažšie sa porovnáva veľkosť jednotlivých častí).** 

### Skladané stĺpcové grafy

Keď zobrazujete podiely a nechcete použiť koláčový graf alebo štandardný stĺpcový graf, môžete ako alternatívu zvážiť aj skladaný stĺpcový graf.

Ak by sme chceli reprodukovať vyššie uvedené koláčové grafy, mohli by sme napísať:



In [ ]:
df_percentages = df["Sex"].value_counts(normalize=True) * 100
pd.DataFrame(df_percentages).T.plot(kind='bar', stacked=True)
plt.grid(ls='--')
plt.gca().set_axisbelow(True)
plt.ylabel("percentages")
plt.savefig("output/stacked_bar_plot_sex.svg", bbox_inches="tight", pad_inches=0)

In [ ]:
# the columns to plot
cols_to_plot = categorical_inputs + [output]

# number of columns and rows in the subplots grid
ncols=3
nrows=int(np.ceil(len(cols_to_plot)/ncols))
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(12, 8))

# plot each column in a separate subplot
for col, ax in zip(cols_to_plot, np.ravel(axes)):
    df_percentages = df[col].value_counts(normalize=True) * 100
    pd.DataFrame(df_percentages).T.plot(kind='bar', stacked=True, ax=ax)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.grid(ls='--')
    ax.set_axisbelow(True)
    
# to hide unused axes
for ax in np.ravel(axes)[len(cols_to_plot):]:
    ax.axis('off')

plt.savefig("output/stacked_bar_plot_grid.svg", bbox_inches="tight", pad_inches=0)

Prirodzene, skladaný stĺpcový graf môžete vytvoriť pomocou akýchkoľvek dát – nemusí ísť o podiely. Mohli by sme napr. vytvoriť kontingenčnú tabuľku, kde by sme mali miesto nalodenia v riadkoch, pohlavie cestujúceho v stĺpcoch a hodnoty by boli počty preživších pasažierov:



In [ ]:
df_pivot = pd.pivot_table(df, index="Embarked", columns="Sex", values="Survived", aggfunc='sum')
df_pivot

Z tejto kontingenčnej tabuľky by sme potom mohli vytvoriť skladaný stĺpcový graf jednoducho takto:



In [ ]:
df_pivot.plot(kind='bar', stacked=True)
plt.grid(ls='--')
plt.gca().set_axisbelow(True)
plt.savefig("output/stacked_bar_plot_pivot.svg", bbox_inches="tight", pad_inches=0)

### Iné grafy

Existujú aj ďalšie, pokročilejšie typy grafov, ktoré môžu byť tiež užitočné v rámci exploratívnej analýzy. Niektorým z nich sa budeme venovať v neskorších moduloch. Ide napr. o:

* Zníženie rozmeru pôvodných dát (napr. pomocou PCA alebo UMAP) a ich vykreslenie v 2D;
* Použitie hierarchického zhlukovania a zobrazenie výsledného dendrogramu (samostatne alebo ako súčasť heatmapy);
* ...
